
<img src="https://lies.mat.uc.cl/wp-content/uploads/2017/09/logo-uc-lineal-azul.png" width="200px" alt="puc logo">
<hr>
<h2 align="center"> A Bayesian Approach to select machine learning models in astronomy  </h2>
<h3 align="left"> Francisco Pérez Galarce </h3>
<h3 align="left"> Advisor: Karim Pichara</h3>
<h3 align="left"> Doctorado en Ciencias de la Ingeniería </h3>
<hr>



*Abstract*: 

In many scientific disciplines and practitioner contexts, the selection of the best mathematical model to represent a real phenomenon is a relevant problem. Typically, two main criteria are considered, the goodness of fit and the model complexity. In the machine learning field, the most common strategies for model comparison are related to cross-validation. These techniques have two important drawbacks, they do not embody the principle of parsimony (also known as Occam's razor principle) and their performance decrease when the sample size is smaller. One way to tackle these limitations is through Bayesian statistic that offers a direct way to compare models through the marginal likelihood function. This function associates the parsimony principle and the goodness of fit. However, this approach has two relevant challenges: (i) the function estimation is very complex when it is worked with high dimensional data and (ii) we must model the phenomenon as a Bayesian model, what sometimes it requires a deep understanding of the Bayesian statistic theory. 

In this research project, it is proposed to apply the marginal likelihood function in the comparison of machine learning models in astronomy. Additionally, the performance of this approach will be compared with traditional methods based on cross-validation. In this talk, will be presented experiments comparing these approaches in basic classification methods in a database of variable stars. 
    

In [1]:
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
import  sklearn.linear_model as linearModel
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
import seaborn 
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')
import timeit
import pymc3 as pm
import matplotlib.lines as mlines
from sklearn import svm
from sklearn import neural_network.MLPClassifier as MLP
np.random.seed(1)
from itertools import product, combinations

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
import bridgeSampling as bs  # it contains a method to estimate the marginal likelihood according to the Bridge Sampling approach
import utilFunctions as ut   # it has different methods to handle and plot data 
import BayesianModels as bm  # it has methods to build and train bayesian model (Logistic Regression and Neural Nets) 
import experiments as exp  

### Select Survey: OGLE, GAIA, VVV, WISE

In [3]:
Data, ID, Class_col, Classes = ut.Initialize(survey='GAIA')

Running GAIA
The dataset contains: 10000 samples


### Define input, it can be PCA or the Most important features 

In [4]:
PCA_ = True  

if PCA_ == False:
    RelevantFeatures = ['ogle_id','PeriodLS','CAR_tau','CAR_tmean', 'Skew', 'Psi_CS', 'Psi_eta', 
                        'SlottedA_length','StetsonK','StetsonK_AC', 'class_name']
    Data =  ut.MostImportanFeature(Data, RelevantFeatures)

In [5]:
Classes

array(['MIRA_SR', 'RRAB', 'CEP', 'RRC', 'DSCT_SXPHE', 'T2CEP', 'RRD',
       'ACEP', 'ARRD'], dtype=object)

# One-vs-one classification

Build $N$($N$ −1) classifiers, one classifier to distinguish each pair of classes $i$ and $j$. Let $f_{ij}$ be the classifier where class $i$ were positive examples and class $j$ were negative.

$  f(x) = \arg \max_i (\sum_j f_{ij} (x)) $

In [10]:
burn_in_ = 100
test = 0.2 
obs_fold = 100
biased, DropEasy, prior = False, False, False
alpha = [0.8]#[0.9+np.round_(i/100,2) for i in range(0,10)]
results = []
summaryAcurracy = []
label1 = Classes[5]
label2 = Classes[0]
for (label1, label2) in list(combinations(Classes, 2)): 
    for test in [i/100 for i in range(1,10)]:
        for k in [6]:
            for prior in [False]:
                try:
                    print('Biased sample: ', biased)
                    #print('alpha: ', a)
                    print('Drop Easier instances:', DropEasy)
                    print('Modfied prior', prior)
                    print(label1, label2)
                    results_summary, model_ogle, trace, dataTrain, dataTest = exp.runExperiments(
                        Data,  class_1= label1, class_2=label2, components= [k], method=[7], 
                        size=[test], ml= False, fit_iterations= 500, name_class_col_= Class_col,                                                          id_col_=ID, biasedSplit = biased, 
                                                                                                                     ModifiedPrior = prior, DropEasy_ =  DropEasy, alpha_ = 0.9, 
                                                                                                                    PCA_ = PCA_, modeltoFit = 'RL')
                    )
                    label = dataTrain['label']
                    labelTest = dataTest['label']
                    del dataTrain['label']
                    del dataTest['label']

                    #r = ut.get_z(dataTrain, trace = trace, burn_in = 500)
                    #predictions_1_Train = ut.logistic_function_(r).mean(axis=1)>0.5
                    acc1 = 0 #accuracy_score(label, predictions_1_Train, normalize=True)
                    print('Accuracy train: ', acc1)

                    #r = ut.get_z(dataTest, trace = trace, burn_in = 500)
                    #predictions_1_Test = ut.logistic_function_(r).mean(axis=1)>0.5
                    acc2 = 0 #accuracy_score(labelTest, predictions_1_Test, normalize=True)
                    print('Accuracy test: ', acc2)



                    n_split_train = 10 #int(dataTrain.shape[0]/obs_fold)
                    n_split_test = 10 #int(dataTest.shape[0]/ obs_fold)
                    #clf1 = linearModel.LogisticRegression(C=1.0)
                    clf1 = RandomForestClassifier()
                    acc_kfold, acc_kfold_Test = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)



                    clf1 = linearModel.LogisticRegression(C=1.0)
                    acc_kfold2, acc_kfold_Test2 = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)

                    clf1 = svm.SVC()
                    acc_kfold3, acc_kfold_Test3 = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)

                    print('Accuracy train RF', np.mean(acc_kfold))
                    print('Accuracy test RF', np.mean(acc_kfold_Test))
                    print('Accuracy train LR', np.mean(acc_kfold2))
                    print('Accuracy test LR', np.mean(acc_kfold_Test2))
                    print('Accuracy train SVM', np.mean(acc_kfold3))
                    print('Accuracy test SVM', np.mean(acc_kfold_Test3))
                    summaryAcurracy.append([k, test, label1, label2, label.shape[0],labelTest.shape[0], np.mean(acc_kfold), np.mean(acc_kfold_Test), np.mean(acc_kfold2), np.mean(acc_kfold_Test2), np.mean(acc_kfold3), np.mean(acc_kfold_Test3), biased, DropEasy, prior])
                    print('-----------------------------------------------------------------------------------------------')
                    #results.append([results_summary, accuracy_score(label, predictions_1_Train, normalize=True), accuracy_score(labelTest, predictions_1_Test, normalize=True), np.mean(acc_kfold), np.mean(acc_kfold_Test), acc_kfold, acc_kfold_Test])
                except: 
                    print('Error')
                
                
file = pd.DataFrame(summaryAcurracy, columns= ['Components','alpha', 'label1', 'label2','size_train', 'Sizetest', 'accTrainRF','accTestRF', 'accTrainRL', 'accTestRL', 'accTrainSVM', 'accTestSVM', 'biased', 'Drop Easier obs', 'Modified Prior'])
file.to_csv('Size_delta_complete_GAIA_RandomForest3.csv')
file

Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_NC VVV_v3_RRc
Running Define TrainSet
(10000, 62)
Test size:  9900
Shape training:  (100, 60)
Shape testing:  (9900, 60)
Running experiments
(100, 60)
(9900, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 476.31it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.871428571429
Accuracy test RF 0.701224269207
Accuracy train LR 0.925
Accuracy test LR 0.945794331537
Accuracy train SVM 0.925
Accuracy test SVM 0.945794331537
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_NC VVV_v3_RRc
Running Define TrainSet
(10000, 62)
Test size:  9800
Shape training:  (200, 60)
Shape testing:  (9800, 60)
Running experiments
(200, 60)
(9800, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 447.08it/s]
The gelman-rubin statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.904642857143
Accuracy test RF 0.944939386178
Accuracy train LR 0.925595238095
Accuracy test LR 0.945961710846
Accuracy train SVM 0.925595238095
Accuracy test SVM 0.945961710846
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_NC VVV_v3_RRc
Running Define TrainSet
(10000, 62)
Test size:  9700
Shape training:  (300, 60)
Shape testing:  (9700, 60)
Running experiments
(300, 60)
(9700, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


...

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 618.12it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.966666666667
Accuracy test RF 0.982875511397
Accuracy train LR 0.966666666667
Accuracy test LR 0.982875511397
Accuracy train SVM 0.966666666667
Accuracy test SVM 0.982875511397
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_ELL VVV_v3_OSARG
Running Define TrainSet
(10000, 62)
Test size:  9100
Shape training:  (900, 60)
Shape testing:  (9100, 60)
Running experiments
(900, 60)
(9100, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 590.96it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.937142857143
Accuracy test RF 0.98275862069
Accuracy train LR 0.957142857143
Accuracy test LR 0.984482758621
Accuracy train SVM 0.957142857143
Accuracy test SVM 0.984482758621
-----------------------------------------------------------------------------------------------


,Components,alpha,label1,label2,size_train,Sizetest,accTrainRF,accTestRF,accTrainRL,accTestRL,accTrainSVM,accTestSVM,biased,Drop Easier obs,Modified Prior
0,6,0.01,VVV_v3_NC,VVV_v3_RRc,76,6918,0.871429,0.701224,0.925000,0.945794,0.925000,0.945794,False,False,False
1,6,0.02,VVV_v3_NC,VVV_v3_RRc,147,6847,0.904643,0.944939,0.925595,0.945962,0.925595,0.945962,False,False,False
2,6,0.03,VVV_v3_NC,VVV_v3_RRc,219,6775,0.932731,0.942288,0.932317,0.945978,0.932317,0.945978,False,False,False
3,6,0.04,VVV_v3_NC,VVV_v3_RRc,291,6703,0.914688,0.938982,0.938374,0.945846,0.938374,0.945846,False,False,False
4,6,0.05,VVV_v3_NC,VVV_v3_RRc,368,6626,0.934835,0.944613,0.945646,0.945518,0.945646,0.945518,False,False,False
5,6,0.06,VVV_v3_NC,VVV_v3_RRc,435,6559,0.928902,0.933224,0.944972,0.945572,0.944972,0.945572,False,False,False
6,6,0.07,VVV_v3_NC,VVV_v3_RRc,496,6498,0.935592,0.855344,0.947673,0.945369,0.947673,0.945369,False,False,False
7,6,0.08,VVV_v3_NC,VVV_v3_RRc,561,6433,0.948296,0.928652,0.953782,0.944817,0.953782,0.944817,False,False,False
8,6,0.09,VVV_v3_NC,VVV_v3_RRc,631,6363,0.947740,0.895806,0.955676,0.944524,0.955676,0.944524,False,False,False
9,6,0.01,VVV_v3_NC,VVV_v3_C,80,8022,0.837500,0.785715,0.875000,0.815632,0.875000,0.815632,False,False,False


In [11]:
Data, ID, Class_col, Classes = ut.Initialize(survey='OGLE')
burn_in_ = 100
test = 0.2 
obs_fold = 100
biased, DropEasy, prior = False, False, False
alpha = [0.8]#[0.9+np.round_(i/100,2) for i in range(0,10)]
results = []
summaryAcurracy = []
#label1 = Classes[5]
#label2 = Classes[0]
for (label1, label2) in list(combinations(Classes, 2)): 
    for test in [i/100 for i in range(1,10)]:
        for k in [6]:
            for prior in [False]:
                try:
                    print('Biased sample: ', biased)
                    #print('alpha: ', a)
                    print('Drop Easier instances:', DropEasy)
                    print('Modfied prior', prior)
                    print(label1, label2)
                    results_summary, model_ogle, trace, dataTrain, dataTest = exp.runExperiments(Data,  class_1= label1, class_2=label2, components= [k],
                                                                                                                     method=[7], size=[test], ml= False, 
                                                                                                                     fit_iterations= 500, name_class_col_= Class_col, 
                                                                                                                     id_col_=ID, biasedSplit = biased, 
                                                                                                                     ModifiedPrior = prior, DropEasy_ =  DropEasy, alpha_ = 0.9, 
                                                                                                                    PCA_ = PCA_, modeltoFit = 'RL')
                    label = dataTrain['label']
                    labelTest = dataTest['label']
                    del dataTrain['label']
                    del dataTest['label']

                    #r = ut.get_z(dataTrain, trace = trace, burn_in = 500)
                    #predictions_1_Train = ut.logistic_function_(r).mean(axis=1)>0.5
                    acc1 = 0 #accuracy_score(label, predictions_1_Train, normalize=True)
                    print('Accuracy train: ', acc1)

                    #r = ut.get_z(dataTest, trace = trace, burn_in = 500)
                    #predictions_1_Test = ut.logistic_function_(r).mean(axis=1)>0.5
                    acc2 = 0 #accuracy_score(labelTest, predictions_1_Test, normalize=True)
                    print('Accuracy test: ', acc2)



                    n_split_train = 10 #int(dataTrain.shape[0]/obs_fold)
                    n_split_test = 10 #int(dataTest.shape[0]/ obs_fold)
                    #clf1 = linearModel.LogisticRegression(C=1.0)
                    clf1 = RandomForestClassifier()
                    acc_kfold, acc_kfold_Test = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)



                    clf1 = linearModel.LogisticRegression(C=1.0)
                    acc_kfold2, acc_kfold_Test2 = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)

                    clf1 = svm.SVC()
                    acc_kfold3, acc_kfold_Test3 = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)

                    print('Accuracy train RF', np.mean(acc_kfold))
                    print('Accuracy test RF', np.mean(acc_kfold_Test))
                    print('Accuracy train LR', np.mean(acc_kfold2))
                    print('Accuracy test LR', np.mean(acc_kfold_Test2))
                    print('Accuracy train SVM', np.mean(acc_kfold3))
                    print('Accuracy test SVM', np.mean(acc_kfold_Test3))
                    summaryAcurracy.append([k, test, label1, label2, label.shape[0],labelTest.shape[0], np.mean(acc_kfold), np.mean(acc_kfold_Test), np.mean(acc_kfold2), np.mean(acc_kfold_Test2), np.mean(acc_kfold3), np.mean(acc_kfold_Test3), biased, DropEasy, prior])
                    print('-----------------------------------------------------------------------------------------------')
                    #results.append([results_summary, accuracy_score(label, predictions_1_Train, normalize=True), accuracy_score(labelTest, predictions_1_Test, normalize=True), np.mean(acc_kfold), np.mean(acc_kfold_Test), acc_kfold, acc_kfold_Test])
                except: 
                    print('Error')
                
                
file = pd.DataFrame(summaryAcurracy, columns= ['Components','alpha', 'label1', 'label2','size_train', 'Sizetest', 'accTrainRF','accTestRF', 'accTrainRL', 'accTestRL', 'accTrainSVM', 'accTestSVM', 'biased', 'Drop Easier obs', 'Modified Prior'])
file.to_csv('Size_delta_complete_OGLE_2.csv')
file

Running OGLE
The dataset contains: 5297 samples
Biased sample:  False
Drop Easier instances: False
Modfied prior False
LPV RRLYR
Running Define TrainSet
(5297, 62)
Test size:  5244
Shape training:  (52, 60)
Shape testing:  (5245, 60)
Running experiments
(52, 60)
(5245, 60)
6
Index(['class_name', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 677.99it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.9
Accuracy test RF 0.665042324411
Accuracy train LR 0.866666666667
Accuracy test LR 0.739001372684
Accuracy train SVM 0.783333333333
Accuracy test SVM 0.668262411348
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
LPV RRLYR
Running Define TrainSet
(5297, 62)
Test size:  5191
Shape training:  (105, 60)
Shape testing:  (5192, 60)
Running experiments
(105, 60)
(5192, 60)
6
Index(['class_name', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 645.38it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.858333333333
Accuracy test RF 0.702361800791
Accuracy train LR 0.95
Accuracy test LR 0.826949477528
Accuracy train SVM 0.916666666667
Accuracy test SVM 0.82803643405
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
LPV RRLYR
Running Define TrainSet
(5297, 62)
Test size:  5138
Shape training:  (158, 60)
Shape testing:  (5139, 60)
Running experiments
(158, 60)
(5139, 60)
6
Index(['class_name', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


...

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 448.48it/s]
The gelman-rubin statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.676428571429
Accuracy test RF 0.629505094465
Accuracy train LR 0.611488095238
Accuracy test LR 0.5192149792
Accuracy train SVM 0.573571428571
Accuracy test SVM 0.506365954368
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
DSCT EB
Running Define TrainSet
(5297, 62)
Test size:  4820
Shape training:  (476, 60)
Shape testing:  (4821, 60)
Running experiments
(476, 60)
(4821, 60)
6
Index(['class_name', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 419.53it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.659722222222
Accuracy test RF 0.617813969637
Accuracy train LR 0.589338235294
Accuracy test LR 0.536622163305
Accuracy train SVM 0.601797385621
Accuracy test SVM 0.504437711075
-----------------------------------------------------------------------------------------------


,Components,alpha,label1,label2,size_train,Sizetest,accTrainRF,accTestRF,accTrainRL,accTestRL,accTrainSVM,accTestSVM,biased,Drop Easier obs,Modified Prior
0,6,0.01,LPV,RRLYR,19,1866,0.900000,0.665042,0.866667,0.739001,0.783333,0.668262,False,False,False
1,6,0.02,LPV,RRLYR,30,1855,0.858333,0.702362,0.950000,0.826949,0.916667,0.828036,False,False,False
2,6,0.03,LPV,RRLYR,50,1835,0.878333,0.744969,0.878333,0.856122,0.875000,0.943308,False,False,False
3,6,0.04,LPV,RRLYR,75,1810,0.958929,0.720991,0.932143,0.796653,0.894246,0.842525,False,False,False
4,6,0.05,LPV,RRLYR,93,1792,0.958889,0.809177,0.891944,0.930787,0.861944,0.848790,False,False,False
5,6,0.06,LPV,RRLYR,116,1769,0.913485,0.785756,0.921061,0.925965,0.819545,0.846261,False,False,False
6,6,0.07,LPV,RRLYR,138,1747,0.933993,0.805349,0.941758,0.898110,0.826227,0.866599,False,False,False
7,6,0.08,LPV,RRLYR,152,1733,0.940655,0.829767,0.921012,0.925555,0.830000,0.845942,False,False,False
8,6,0.09,LPV,RRLYR,171,1714,0.929412,0.869345,0.894444,0.943988,0.835621,0.836062,False,False,False
9,6,0.01,LPV,CEP,26,2354,0.900000,0.800320,0.733333,0.837708,0.683333,0.827518,False,False,False


In [12]:
Data, ID, Class_col, Classes = ut.Initialize(survey='VVV')
burn_in_ = 100
test = 0.2 
obs_fold = 100
biased, DropEasy, prior = False, False, False
alpha = [0.8]#[0.9+np.round_(i/100,2) for i in range(0,10)]
results = []
summaryAcurracy = []
#label1 = Classes[5]
#label2 = Classes[0]
for (label1, label2) in list(combinations(Classes, 2)): 
    for test in [i/100 for i in range(1,10)]:
        for k in [6]:
            for prior in [False]:
                try:
                    print('Biased sample: ', biased)
                    #print('alpha: ', a)
                    print('Drop Easier instances:', DropEasy)
                    print('Modfied prior', prior)
                    print(label1, label2)
                    results_summary, model_ogle, trace, dataTrain, dataTest = exp.runExperiments(Data,  class_1= label1, class_2=label2, components= [k],
                                                                                                                     method=[7], size=[test], ml= False, 
                                                                                                                     fit_iterations= 500, name_class_col_= Class_col, 
                                                                                                                     id_col_=ID, biasedSplit = biased, 
                                                                                                                     ModifiedPrior = prior, DropEasy_ =  DropEasy, alpha_ = 0.9, 
                                                                                                                    PCA_ = PCA_, modeltoFit = 'RL')
                    label = dataTrain['label']
                    labelTest = dataTest['label']
                    del dataTrain['label']
                    del dataTest['label']

                    #r = ut.get_z(dataTrain, trace = trace, burn_in = 500)
                    #predictions_1_Train = ut.logistic_function_(r).mean(axis=1)>0.5
                    acc1 = 0 #accuracy_score(label, predictions_1_Train, normalize=True)
                    print('Accuracy train: ', acc1)

                    #r = ut.get_z(dataTest, trace = trace, burn_in = 500)
                    #predictions_1_Test = ut.logistic_function_(r).mean(axis=1)>0.5
                    acc2 = 0 #accuracy_score(labelTest, predictions_1_Test, normalize=True)
                    print('Accuracy test: ', acc2)



                    n_split_train = 10 #int(dataTrain.shape[0]/obs_fold)
                    n_split_test = 10 #int(dataTest.shape[0]/ obs_fold)
                    #clf1 = linearModel.LogisticRegression(C=1.0)
                    clf1 = RandomForestClassifier()
                    acc_kfold, acc_kfold_Test = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)



                    clf1 = linearModel.LogisticRegression(C=1.0)
                    acc_kfold2, acc_kfold_Test2 = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)

                    clf1 = svm.SVC()
                    acc_kfold3, acc_kfold_Test3 = ut.KFold(dataTrain, label, dataTest, labelTest, n_split_test, n_split_train, clf1)

                    print('Accuracy train RF', np.mean(acc_kfold))
                    print('Accuracy test RF', np.mean(acc_kfold_Test))
                    print('Accuracy train LR', np.mean(acc_kfold2))
                    print('Accuracy test LR', np.mean(acc_kfold_Test2))
                    print('Accuracy train SVM', np.mean(acc_kfold3))
                    print('Accuracy test SVM', np.mean(acc_kfold_Test3))
                    summaryAcurracy.append([k, test, label1, label2, label.shape[0],labelTest.shape[0], np.mean(acc_kfold), np.mean(acc_kfold_Test), np.mean(acc_kfold2), np.mean(acc_kfold_Test2), np.mean(acc_kfold3), np.mean(acc_kfold_Test3), biased, DropEasy, prior])
                    print('-----------------------------------------------------------------------------------------------')
                    #results.append([results_summary, accuracy_score(label, predictions_1_Train, normalize=True), accuracy_score(labelTest, predictions_1_Test, normalize=True), np.mean(acc_kfold), np.mean(acc_kfold_Test), acc_kfold, acc_kfold_Test])
                except: 
                    print('Error')
                
                
file = pd.DataFrame(summaryAcurracy, columns= ['Components','alpha', 'label1', 'label2','size_train', 'Sizetest', 'accTrainRF','accTestRF', 'accTrainRL', 'accTestRL', 'accTrainSVM', 'accTestSVM', 'biased', 'Drop Easier obs', 'Modified Prior'])
file.to_csv('Size_delta_complete_VVV_2.csv')
file

Running VVV
The dataset contains: 10000 samples
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_NC VVV_v3_RRc
Running Define TrainSet
(10000, 62)
Test size:  9900
Shape training:  (100, 60)
Shape testing:  (9900, 60)
Running experiments
(100, 60)
(9900, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 569.31it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.928571428571
Accuracy test RF 0.951797622816
Accuracy train LR 0.942857142857
Accuracy test LR 0.951797622816
Accuracy train SVM 0.942857142857
Accuracy test SVM 0.951797622816
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_NC VVV_v3_RRc
Running Define TrainSet
(10000, 62)
Test size:  9800
Shape training:  (200, 60)
Shape testing:  (9800, 60)
Running experiments
(200, 60)
(9800, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 442.68it/s]
The gelman-rubin statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.888864468864
Accuracy test RF 0.949964207405
Accuracy train LR 0.919084249084
Accuracy test LR 0.952298272619
Accuracy train SVM 0.919084249084
Accuracy test SVM 0.952298272619
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_NC VVV_v3_RRc
Running Define TrainSet
(10000, 62)
Test size:  9700
Shape training:  (300, 60)
Shape testing:  (9700, 60)
Running experiments
(300, 60)
(9700, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


...

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 582.88it/s]
The gelman-rubin statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.92619047619
Accuracy test RF 0.819883889695
Accuracy train LR 0.942857142857
Accuracy test LR 0.98084179971
Accuracy train SVM 0.942857142857
Accuracy test SVM 0.98084179971
-----------------------------------------------------------------------------------------------
Biased sample:  False
Drop Easier instances: False
Modfied prior False
VVV_v3_ELL VVV_v3_OSARG
Running Define TrainSet
(10000, 62)
Test size:  9100
Shape training:  (900, 60)
Shape testing:  (9100, 60)
Running experiments
(900, 60)
(9100, 60)
6
Index(['Class', 'PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'], dtype='object')
------- Metropolis--------


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [PC5]
>Metropolis: [PC4]
>Metropolis: [PC3]
>Metropolis: [PC2]
>Metropolis: [PC1]
>Metropolis: [PC0]
>Metropolis: [Intercept]
100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 574.95it/s]
The gelman-rubin statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


return the last model and trace
Accuracy train:  0
Accuracy test:  0
Training...
Testing...
Training...
Testing...
Training...
Testing...
Accuracy train RF 0.908333333333
Accuracy test RF 0.893476621418
Accuracy train LR 0.95
Accuracy test LR 0.980618401207
Accuracy train SVM 0.95
Accuracy test SVM 0.980618401207
-----------------------------------------------------------------------------------------------


,Components,alpha,label1,label2,size_train,Sizetest,accTrainRF,accTestRF,accTrainRL,accTestRL,accTrainSVM,accTestSVM,biased,Drop Easier obs,Modified Prior
0,6,0.01,VVV_v3_NC,VVV_v3_RRc,61,6929,0.928571,0.951798,0.942857,0.951798,0.942857,0.951798,False,False,False
1,6,0.02,VVV_v3_NC,VVV_v3_RRc,135,6855,0.888864,0.949964,0.919084,0.952298,0.919084,0.952298,False,False,False
2,6,0.03,VVV_v3_NC,VVV_v3_RRc,208,6782,0.889957,0.938073,0.918766,0.952669,0.918766,0.952669,False,False,False
3,6,0.04,VVV_v3_NC,VVV_v3_RRc,265,6725,0.894160,0.882384,0.924501,0.952714,0.924501,0.952714,False,False,False
4,6,0.05,VVV_v3_NC,VVV_v3_RRc,343,6647,0.906776,0.943886,0.927375,0.952912,0.927375,0.952912,False,False,False
5,6,0.06,VVV_v3_NC,VVV_v3_RRc,405,6585,0.908765,0.945483,0.931024,0.952923,0.931024,0.952923,False,False,False
6,6,0.07,VVV_v3_NC,VVV_v3_RRc,470,6520,0.897845,0.942488,0.929903,0.953222,0.929903,0.953222,False,False,False
7,6,0.08,VVV_v3_NC,VVV_v3_RRc,536,6454,0.921733,0.920975,0.932914,0.953208,0.932914,0.953208,False,False,False
8,6,0.09,VVV_v3_NC,VVV_v3_RRc,614,6376,0.923453,0.947931,0.934849,0.953263,0.934849,0.953263,False,False,False
9,6,0.01,VVV_v3_NC,VVV_v3_C,75,8097,0.650595,0.706680,0.763095,0.814499,0.763095,0.814499,False,False,False
